In [1]:
from llama_index.core import Document, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb

# Inde metadata

https://docs.llamaindex.ai/en/stable/understanding/loading/loading/

https://docs.llamaindex.ai/en/stable/examples/vector_stores/ChromaIndexDemo/

https://docs.llamaindex.ai/en/stable/understanding/querying/querying/

https://docs.llamaindex.ai/en/stable/module_guides/querying/retriever/

In [2]:
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")

In [3]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Pkalanski\Anaconda3\envs\llm-env\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Pkalanski\AppData\Local\llama_index\models--BAAI--bge-base-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [4]:
hf_models = 'hf1,hf2,hf3'
bedrock_models = 'b1,b2'

models = {
    "hf": hf_models.split(',') if hf_models else [],
    "bedrock": bedrock_models.split(',') if bedrock_models else [],
}

for provider in models:
    for model_id in models[provider]:
        print(provider, model_id)

hf hf1
hf hf2
hf hf3
bedrock b1
bedrock b2


In [5]:
documents = [
    Document(text="main.customers", metadata= {'description': "This table has basic information about a customer, as well as some derived facts based on a customer's orders", 'columns': 'customer_id,first_name,last_name,first_order,most_recent_order,number_of_orders,total_order_amount'} ),
    Document(text="main.orders", metadata= {'description': 'This table has basic information about orders, as well as some derived facts based on payments', 'columns': 'order_id,customer_id,order_date,status,amount,credit_card_amount,coupon_amount,bank_transfer_amount,gift_card_amount'} ),
    Document(text="main.stg_customers", metadata= {'description': '', 'columns': 'customer_id'} ),
    Document(text="main.stg_orders", metadata= {'description': '', 'columns': 'order_id,status'} ),
    Document(text="main.stg_payments", metadata= {'description': '', 'columns': 'payment_id,payment_method'} ),
]

In [6]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

In [7]:
from llama_index.core.vector_stores.types import VectorStoreQuery
q = index.vector_store.aquery(VectorStoreQuery(
    query_str="customer"
))

In [8]:
embed_model.get_query_embedding("customer")

[0.04045151546597481,
 -0.012893866747617722,
 -0.015111207962036133,
 0.004921073094010353,
 0.015513082966208458,
 -0.0205589160323143,
 0.010152550414204597,
 -0.0032247796189039946,
 0.026975609362125397,
 0.03301788493990898,
 -0.014193070121109486,
 0.00835871696472168,
 -0.010251120664179325,
 -0.0025073527358472347,
 -0.01074238121509552,
 -0.01812760904431343,
 -0.003558783559128642,
 0.007882471196353436,
 -0.013915264047682285,
 -0.04287642240524292,
 0.0007755251135677099,
 0.015087230131030083,
 -0.027032693848013878,
 0.014136053621768951,
 -0.028909843415021896,
 0.017868727445602417,
 -0.024475548416376114,
 -0.02005559764802456,
 0.031161800026893616,
 0.03043859265744686,
 -0.044531118124723434,
 0.0167362242937088,
 0.010496780276298523,
 -0.053803157061338425,
 0.014004425145685673,
 -0.016234593465924263,
 0.018087266013026237,
 -0.036834366619586945,
 -0.0575566440820694,
 -0.050886187702417374,
 -0.03445335477590561,
 -0.01709936000406742,
 0.08277081698179245,
 

# Save to disk

In [9]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [10]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

# Load from disk

In [11]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

# Query

In [12]:
chroma_collection.count()

5

In [13]:
query_response = chroma_collection.query(
    query_embeddings=[
        embed_model.get_query_embedding("customer")
    ],
    n_results=10,
)
# https://docs.trychroma.com/getting-started

Number of requested results 10 is greater than number of elements in index 5, updating n_results = 5


In [14]:
query_response

{'ids': [['bd972ef8-89f5-40d8-8c3f-3afdab30dc13',
   '6f836389-0d06-41e3-b46f-a40bafe6857e',
   'c5e09052-51ce-40d2-b89a-5d3c76e2ab4d',
   '73a6337b-732d-49e3-8bda-92fd8fc66c42',
   'aa9070fd-262e-4841-b266-c7d29562ee4e']],
 'distances': [[0.5881536435133103,
   0.683094389258029,
   0.8614222646841738,
   0.877496233888225,
   0.8890770438712547]],
 'metadatas': [[{'_node_content': '{"id_": "bd972ef8-89f5-40d8-8c3f-3afdab30dc13", "embedding": null, "metadata": {"description": "", "columns": "customer_id"}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {"1": {"node_id": "25559a4e-dd69-4345-a89e-d4540817d01e", "node_type": "4", "metadata": {"description": "", "columns": "customer_id"}, "hash": "0c08895b73233902cae936b21232cb03eea08d9145f2a379adaa7e090d205ace", "class_name": "RelatedNodeInfo"}}, "text": "", "start_char_idx": 0, "end_char_idx": 18, "text_template": "{metadata_str}\\n\\n{content}", "metadata_template": "{key}: {value}", "metadata_se

In [15]:
query_response.keys()

dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data'])

In [16]:
query_response['distances']

[[0.5881536435133103,
  0.683094389258029,
  0.8614222646841738,
  0.877496233888225,
  0.8890770438712547]]

In [17]:
query_response['metadatas'][0][1]['_node_content']

'{"id_": "6f836389-0d06-41e3-b46f-a40bafe6857e", "embedding": null, "metadata": {"description": "This table has basic information about a customer, as well as some derived facts based on a customer\'s orders", "columns": "customer_id,first_name,last_name,first_order,most_recent_order,number_of_orders,total_order_amount"}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {"1": {"node_id": "f89a5466-af7c-4f71-92dd-75167879a77d", "node_type": "4", "metadata": {"description": "This table has basic information about a customer, as well as some derived facts based on a customer\'s orders", "columns": "customer_id,first_name,last_name,first_order,most_recent_order,number_of_orders,total_order_amount"}, "hash": "8fab84d152d6deede018715ffdbe8949e776d9513f9e2a341f2c5d1b625d57d5", "class_name": "RelatedNodeInfo"}}, "text": "", "start_char_idx": 0, "end_char_idx": 14, "text_template": "{metadata_str}\\n\\n{content}", "metadata_template": "{key}: {value}", "met

In [18]:
import json

In [19]:
nc = json.loads(query_response['metadatas'][0][1]['_node_content'])
nc

{'id_': '6f836389-0d06-41e3-b46f-a40bafe6857e',
 'embedding': None,
 'metadata': {'description': "This table has basic information about a customer, as well as some derived facts based on a customer's orders",
  'columns': 'customer_id,first_name,last_name,first_order,most_recent_order,number_of_orders,total_order_amount'},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {'1': {'node_id': 'f89a5466-af7c-4f71-92dd-75167879a77d',
   'node_type': '4',
   'metadata': {'description': "This table has basic information about a customer, as well as some derived facts based on a customer's orders",
    'columns': 'customer_id,first_name,last_name,first_order,most_recent_order,number_of_orders,total_order_amount'},
   'hash': '8fab84d152d6deede018715ffdbe8949e776d9513f9e2a341f2c5d1b625d57d5',
   'class_name': 'RelatedNodeInfo'}},
 'text': '',
 'start_char_idx': 0,
 'end_char_idx': 14,
 'text_template': '{metadata_str}\n\n{content}',
 'metadata_template':

In [20]:
nc['metadata']

{'description': "This table has basic information about a customer, as well as some derived facts based on a customer's orders",
 'columns': 'customer_id,first_name,last_name,first_order,most_recent_order,number_of_orders,total_order_amount'}

In [21]:
result = []
for m in query_response['metadatas'][0]:
    nc = json.loads(m['_node_content'])
    node_metadata = nc['metadata']
    result.append(node_metadata['columns'])

result

['customer_id',
 'customer_id,first_name,last_name,first_order,most_recent_order,number_of_orders,total_order_amount',
 'payment_id,payment_method',
 'order_id,status',
 'order_id,customer_id,order_date,status,amount,credit_card_amount,coupon_amount,bank_transfer_amount,gift_card_amount']

In [22]:
import string
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'